In [1]:
import codecs
import numpy as np
from numpy import linalg as LA
from sklearn.metrics.pairwise import cosine_similarity

def load_embeddings_from_np(filename):
    print('loading ...')
    with codecs.open(filename + '.vocab', 'r', 'utf-8') as f_embed:
        vocab = [line.strip() for line in f_embed]
        
    w2i = {w: i for i, w in enumerate(vocab)}
    wv = np.load(filename + '.wv.npy')

    return vocab, wv, w2i

def load_dhdglove(path):
    print('loading ...')
    debiased_embeds = pickle.load(open(path, 'rb'))
    wv = []
    vocab = []
    for w in debiased_embeds:
        wv.append(np.array(debiased_embeds[w]))
        vocab.append(str(w))
        
    w2i = {w: i for i, w in enumerate(vocab)}
    wv = np.array(wv).astype(float)
    print(len(vocab), wv.shape, len(w2i))
        
    return vocab, wv, w2i 

def load_wo_normalize(space, filename, vocab, wv, w2i):
    if filename[-3:]=='txt':
        vocab_muse, wv_muse, w2i_muse = load_embeddings_from_np(filename)
    else:
        vocab_muse, wv_muse, w2i_muse = load_dhdglove(filename)
    vocab[space] = vocab_muse 
    wv[space] = wv_muse
    w2i[space] = w2i_muse
    print('done')


In [2]:
vocab = {}
wv = {}
w2i = {}

load_wo_normalize('bef', 'Gender-Biased Word Relation Task/data/embeddings/glove_wiki_vectors.txt', vocab, wv, w2i)
load_wo_normalize('aft', 'Gender-Biased Word Relation Task/data/embeddings/vectors_hd.txt', vocab, wv, w2i)

loading ...
done
loading ...
done


In [3]:
orig_glove = dict(zip(vocab['bef'], wv['bef']))
post_glove = dict(zip(vocab['aft'], wv['aft']))

In [4]:
filename = 'data/SemBias.txt'

def read_SemBias(filename):
    file_read = open(filename, "r", encoding = 'utf8')
    
    SemBias_list = []
    
    for line in file_read:
        pairs = line.rstrip().split('\t')
        
        line_temp = []
        for p in pairs:
            a, b = p.split(':')
            line_temp.append([a,b])
        
        SemBias_list.append(line_temp)
    
    return SemBias_list

SemBias_task = read_SemBias(filename)

In [5]:
def eval_top(task, wordVecModel_str):
    wordVecModel = eval(wordVecModel_str)
    #vocab = set(list(wordVecModel.keys()))
    result_list = []
    he_we = wordVecModel['he'].reshape(1,-1)
    she_we = wordVecModel['she'].reshape(1,-1)
    
    for line in task:
        temp_score = []
        
        if len(line) != 4:
            print('error')
            
        for pair in line:
            (word_i, word_j) = pair
            current_distance = cosine_similarity(he_we - she_we , wordVecModel[word_i].reshape(1,-1) - wordVecModel[word_j].reshape(1,-1) )        
            temp_score.append(current_distance)
        
        result_list.append(temp_score.index(max(temp_score)))
        
    return result_list

xx_orig = eval_top(SemBias_task, 'orig_glove')
xx_hs = eval_top(SemBias_task, 'post_glove')

In [6]:
list_xx = [xx_orig, xx_hs]

print('Orig gender-definition:')
print('SemBias: ',end =" ")
print(list_xx[0].count(0)/440,end =" ")
print('SemBias (subset): ',end =" ")
print(list_xx[0][-40:].count(0)/40)
      
print('HARD gender-definition:')
print('SemBias: ',end =" ")
print(list_xx[1].count(0)/440,end =" ")
print('SemBias (subset): ',end =" ")
print(list_xx[1][-40:].count(0)/40)

Orig gender-definition:
SemBias:  0.8022727272727272 SemBias (subset):  0.575
HARD gender-definition:
SemBias:  0.8159090909090909 SemBias (subset):  0.525


In [7]:
print('Orig gender-stereotype:')
print('SemBias: ',end =" ")
print(list_xx[0].count(3)/440,end =" ")
print('SemBias (subset): ',end =" ")
print(list_xx[0][-40:].count(3)/40)
      
print('HARD-GloVe gender-stereotype:')
print('SemBias: ',end =" ")
print(list_xx[1].count(3)/440,end =" ")
print('SemBias (subset): ',end =" ")
print(list_xx[1][-40:].count(3)/40)

Orig gender-stereotype:
SemBias:  0.10909090909090909 SemBias (subset):  0.2
HARD-GloVe gender-stereotype:
SemBias:  0.10681818181818181 SemBias (subset):  0.25


In [8]:
print('Orig none:')
print('SemBias: ',end =" ")
print((list_xx[0].count(1)+list_xx[0].count(2))/440,end =" ")
print('SemBias (subset): ',end =" ")
print((list_xx[0][-40:].count(1)+list_xx[0][-40:].count(2))/40)
      
print('HARD-GloVe none:')
print('SemBias: ',end =" ")
print((list_xx[1].count(1)+list_xx[1].count(2))/440,end =" ")
print('SemBias (subset): ',end =" ")
print((list_xx[1][-40:].count(1)+list_xx[1][-40:].count(2))/40)

Orig none:
SemBias:  0.08863636363636364 SemBias (subset):  0.225
HARD-GloVe none:
SemBias:  0.07727272727272727 SemBias (subset):  0.225
